A.S. Lundervold, v111022

> **Note:** This is a short notebook giving a quick taste of a concept that's also covered elsewhere in the course. It should be regarded as extra material. 

# Setup

In [ ]:
%matplotlib inline

import numpy as np, pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path
import seaborn as sns 
import sklearn
from sklearn import datasets

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# or on Kaggle, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

# Imputering

> Hvordan håndtere manglende verdier? 

1. Først må de **detekteres**. I praksis kan manglende verdier være kodet på omtrent hvilken som helst måte. Ofte bruker man blanke verdier, placeholder-verdier (f.eks. -1) eller NaNs (not-a-number).

2. Deretter kan man _imputere_. Det vil si, fylle inn manglende data

# Data

In [ ]:
NB_DIR = Path.cwd()
DATA = NB_DIR/'data'
DATA.mkdir(exist_ok=True)

Vi bruker datasettet fra innlevering 1:

In [ ]:
train = pd.read_csv('https://www.dropbox.com/s/zwrfpg7ww6mj5bz/housing_train_missing.csv?dl=1')
test = pd.read_csv('https://www.dropbox.com/s/03y7zsdbcb4xouw/housing_test_missing.csv?dl=1')

In [ ]:
train.head()

In [ ]:
test.head()

Historien vi skal fortelle om dette har behov for at vi er i en maskinlærings-situasjon med X, y og trenings- og test-data:

In [ ]:
X_train, y_train = train.drop(columns='median_house_value'), train.median_house_value
X_test = test

## 1. Detektere manglende verdier

Det første man typisk undersøker er hvorvidt det er NaN-verdier i datasettet. Hvis en arbeider med en Pandas dataframe er dette enkelt: 

In [ ]:
X_train.head(10)

In [ ]:
X_train.isna()

In [ ]:
X_train.isna().sum()

In [ ]:
plt.figure(figsize=(16,8))
X_train.isna().sum().plot(kind='bar')
plt.show()

Men manglende verdier kan være kodet annerledes. En måte å finne disse er å se på hvilke verdier som finnes i datasettet. 

I vårt tilfelle er features kodet fly-tall og strenger. Manglende verdier kan f.eks. være representert som -1 eller som tomme strenger " ", men vi kan ikke se bort fra at representasjonene av manglende verdier kan være en annen.

In [ ]:
X_train.info()

Hvis features ikke har for mange ulike verdier er det nyttig å telle opp antall instanser som har hver verdi:

In [ ]:
for feature in X_train.columns: 
    print(f"Value count for {feature}")
    with pd.option_context('display.max_rows', None): 
        print(X_train[feature].value_counts())
    print("#"*40)

1227.0     22
761.0      20
850.0      20
1098.0     19
1052.0     19
1086.0     19
782.0      18
926.0      18
855.0      18
804.0      18
891.0      18
1155.0     18
1047.0     17
810.0      17
984.0      17
704.0      17
913.0      17
859.0      17
1301.0     17
928.0      16
899.0      16
848.0      16
1193.0     16
1006.0     16
970.0      16
1208.0     16
1092.0     16
999.0      16
1312.0     16
861.0      16
933.0      16
779.0      16
753.0      16
943.0      16
852.0      16
793.0      16
1158.0     16
986.0      16
1128.0     16
731.0      16
1200.0     16
705.0      16
1005.0     16
1203.0     16
781.0      16
857.0      15
862.0      15
910.0      15
788.0      15
825.0      15
679.0      15
937.0      15
872.0      15
918.0      15
735.0      15
768.0      15
837.0      15
774.0      15
939.0      15
671.0      15
823.0      15
887.0      15
1304.0     15
863.0      15
1074.0     15
973.0      15
1277.0     15
846.0      15
811.0      15
883.0      15
1257.0     15
868.0 

* Vi observerer at verdien -1 opptrer to ganger i `total_bedrooms` og verdien 99999 én gang. Disse verdiene indikerer trolig manglende verdier. 
* I `ocean_proximity` finner vi en tom streng " " én gang. Dette er også trolig en indikasjon på manglende verdi.

Vi gjør det samme med testsettet:

In [ ]:
for feature in X_test.columns: 
    print(f"Value count for {feature}")
    with pd.option_context('display.max_rows', None): 
        print(X_test[feature].value_counts())
    print("#"*40)

287.0     14
373.0     14
342.0     13
420.0     13
324.0     13
559.0     13
390.0     12
328.0     12
399.0     12
431.0     12
347.0     12
313.0     12
348.0     12
278.0     12
428.0     12
280.0     12
419.0     11
438.0     11
322.0     11
272.0     11
361.0     11
292.0     11
364.0     11
366.0     11
359.0     11
224.0     11
423.0     11
488.0     11
282.0     11
360.0     11
408.0     11
307.0     11
375.0     10
314.0     10
353.0     10
227.0     10
417.0     10
294.0     10
269.0     10
497.0     10
262.0     10
461.0     10
365.0     10
498.0     10
406.0     10
395.0     10
321.0     10
264.0     10
285.0     10
508.0     10
378.0     10
435.0     10
372.0     10
396.0     10
202.0     10
662.0     10
323.0      9
356.0      9
275.0      9
380.0      9
250.0      9
312.0      9
397.0      9
301.0      9
524.0      9
338.0      9
371.0      9
466.0      9
351.0      9
271.0      9
309.0      9
246.0      9
487.0      9
493.0      9
339.0      9
346.0      9
606.0      9

* Vi finner verdien -1 i featuren `households` én gang. Indikerer trolig manglende verdi.

### Fiks encoding av manglende verdier

Basert på disse observasjonene så skifter vi encoding av manglende verdier slik at den blir konsistent. (Siden det gjelder såpass få instanser kan vi gjøre dette manuelt)

**Train**

In [ ]:
X_train.loc[X_train.total_bedrooms==-1]

In [ ]:
X_train.loc[X_train.total_bedrooms==99999]

In [ ]:
X_train.loc[504, 'total_bedrooms'] = np.nan
X_train.loc[16272, 'total_bedrooms'] = np.nan
X_train.loc[88, 'total_bedrooms'] = np.nan

In [ ]:
X_train.loc[X_train.ocean_proximity == " "]

In [ ]:
X_train.loc[16489, 'ocean_proximity'] = np.nan

**Test**

In [ ]:
X_test.loc[X_test.households == -1]

In [ ]:
X_test.loc[504, 'households'] = np.nan

**Sjekk**

In [ ]:
X_train.isna().sum()

In [ ]:
X_test.isna().sum()

## 2. Imputere

Det neste blir å erstatte disse NaN-verdiene. Som dere har sett tidligere så er det mange ulike strategier. 

### Enkle strategier

Vi starter med noen enkle strategier: for de numeriske søylene kan vi erstatte verdiene ved å bruke gjennomsnitt, median eller den hyppigste verdien

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
?SimpleImputer

In [ ]:
imp = SimpleImputer(strategy='mean')

Merk at man kan bruke samme strategi for hver feature, eller ulike strategier for ulike features.

For de kategoriske features (`ocean_proximity` i vårt tilfelle) kan vi bruke `most_frequent` som strategi. 

Her er pipelines som behandler numeriske og kategoriske features hver for seg:

In [ ]:
from sklearn.pipeline import make_pipeline

numerical_pipeline = make_pipeline(SimpleImputer(strategy="median"))

categorical_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"),)

Vi kan bruke disse på våre data via en `columns_selector` og en `column_transformer`:

In [ ]:
from sklearn.compose import make_column_selector, make_column_transformer

impute = make_column_transformer(
    (numerical_pipeline, make_column_selector(dtype_include=np.number)),
    (categorical_pipeline, make_column_selector(dtype_include=object)),
)

In [ ]:
X_test.head()

In [ ]:
X_train_imp = impute.fit_transform(X_train)
X_test_imp = impute.transform(X_test) # NB: Note that there's no "fit" for the test data

> ***Hvorfor bruker man `.fit_transform` på treningssettet, men `.transform` på test-settet?***

In [ ]:
X_train_imp = pd.DataFrame(X_train_imp, columns=X_train.columns)
X_test_imp = pd.DataFrame(X_test_imp, columns=X_test.columns)

In [ ]:
X_train_imp.head(10)

Ingen flere missing values:

In [ ]:
X_train_imp.info()

In [ ]:
X_test_imp.info()

I noen situasjoner kan det være nyttig informasjon for prediktive modeller i hvorvidt features mangler eller ikke. Slik informasjon kan bevares ved å sette inn indikator-søyler:

In [ ]:
numerical_pipeline = make_pipeline(SimpleImputer(strategy="median", add_indicator=True))
categorical_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent", add_indicator=True))
impute = make_column_transformer(
    (numerical_pipeline, make_column_selector(dtype_include=np.number)),
    (categorical_pipeline, make_column_selector(dtype_include=object)),
)

In [ ]:
X_train_imp_ind = impute.fit_transform(X_train)

In [ ]:
X_train_imp_ind = pd.DataFrame(X_train_imp_ind)

In [ ]:
X_train_imp_ind.head(10)

### Mer avanserte strategier

En kan være mer spissfindig enn å imputere verdier ved å bruke verdiene til alle instanser. Man kan for eksempel finne hvilke verdier en skal erstatte med ved å bruke kun _lignende_ instanser, istedenfor alle. 

En strategi for dette er å trene en modell som kan gruppere lignende instanser. Et eksempel på dette er såkalte **K nærmeste nabo** eller KNN.  

<img width=60% src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/850px-KnnClassification.svg.png'>

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
?KNNImputer

In [ ]:
knn_imp = KNNImputer()

In [ ]:
X_train[40:50]

In [ ]:
numerical_pipeline = make_pipeline(KNNImputer())
categorical_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"))
impute = make_column_transformer(
    (numerical_pipeline, make_column_selector(dtype_include=np.number)),
    (categorical_pipeline, make_column_selector(dtype_include=object)),
)

In [ ]:
X_train_imp = impute.fit_transform(X_train)
X_train_imp = pd.DataFrame(X_train_imp, columns=X_train.columns)

In [ ]:
X_train_imp[40:50]

### En annen strategi: tren en modell til å imputere

En annen strategi er å trene en regresjonsmodell, for eksempel en RandomForestRegressor, til å predikere manglende verdier fra verdiene som ikke mangler. 

Dette kan gjøres iterativt: i hvert steg velges en feature-søyle til å gi outputs `y`. Deretter trenes en modell på alle features uten manglende verdier til å predikere `y` ved å bruke de instansene der en kjenner verdiene av `y`. Denne modellen kan så brukes til å fylle inn. Dette kan en iterere over alle søylene til en ikke lenger har manglende verdier, og man kan gjøre det om og om igjen til en når et maksimalt antall iterasjoner. 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
from sklearn.ensemble import RandomForestRegressor

estimator = RandomForestRegressor()

In [ ]:
#?IterativeImputer

In [ ]:
X_train_copy = X_train.copy()

In [ ]:
X_train_copy[40:50]

In [ ]:
numerical_pipeline = make_pipeline(IterativeImputer())
categorical_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent", add_indicator=True))
impute = make_column_transformer(
    (numerical_pipeline, make_column_selector(dtype_include=np.number)),
    (categorical_pipeline, make_column_selector(dtype_include=object)),
)

In [ ]:
X_train_imp = impute.fit_transform(X_train)
X_train_imp = pd.DataFrame(X_train, columns=X_train.columns)

In [ ]:
X_train_imp.head()